<a href="https://colab.research.google.com/github/AnjanaGJoseph/Signature_forgery_detection/blob/master/signature_classification_using_siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All imports are defined here:-

In [1]:
import numpy as np 
import pandas as pd
import os

%matplotlib inline
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import matplotlib.pyplot as plt
import torchvision.utils
import random
from PIL import Image
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

In [2]:
os.environ['KAGGLE_USERNAME'] = "anjana22"
os.environ['KAGGLE_KEY'] = "16acf05db0477eb1236698f72ea68121"

In [3]:
!kaggle datasets download -d robinreni/signature-verification-dataset

 99% 593M/601M [00:07<00:00, 85.9MB/s]
100% 601M/601M [00:07<00:00, 82.1MB/s]


In [4]:

!unzip -q signature-verification-dataset.zip

# Defining Training Directories and CSV's:-

In [5]:
train_dir="/content/sign_data/train"
train_csv="/content/sign_data/train_data.csv"
test_csv="/content/sign_data/test_data.csv"
test_dir="/content/sign_data/test"

In [6]:
df_train=pd.read_csv(train_csv)
df_train.sample(10)

,068/09_068.png,068_forg/03_0113068.PNG,1
8210,009/009_03.PNG,009/009_24.PNG,0
5876,022/11_022.png,022/06_022.png,0
2766,004/004_01.PNG,004/004_23.PNG,0
15268,043/02_043.png,043_forg/02_0201043.PNG,1
21385,025/02_025.png,025_forg/03_0116025.PNG,1
1663,038/07_038.png,038_forg/02_0213038.PNG,1
22491,061/10_061.png,061_forg/03_0112061.PNG,1
7636,009/009_19.PNG,009_forg/0123009_03.png,1
21487,025/08_025.png,025/11_025.png,0
5078,012/012_18.PNG,012/012_03.PNG,0


# Here we are seeing that 1 denotes for forged pair and 0 denotes for geniune pair of signatures..

In [7]:

df_test=pd.read_csv(test_csv)
df_test.sample(10)

,068/09_068.png,068_forg/03_0113068.PNG,1
4386,063/08_063.png,063/06_063.png,0
5491,066/09_066.png,066_forg/04_0101066.PNG,1
155,068/04_068.png,068/12_068.png,0
3255,060/01_060.png,060_forg/02_0121060.PNG,1
2565,053/01_053.png,053/04_053.png,0
3439,060/05_060.png,060/01_060.png,0
4435,063/02_063.png,063/08_063.png,0
5199,061/06_061.png,061_forg/02_0102061.PNG,1
2387,053/01_053.png,053_forg/03_0107053.PNG,1
1021,065/02_065.png,065/10_065.png,0


In [8]:
df_train.shape

(23205, 3)

In [9]:
df_test.shape

(5747, 3)

# Making Custom Pytorch Siamese Dataset:-

the ____len____ function which returns the size of the dataset, and

the ____getitem____ function which returns a sample from the dataset given an index.

In [10]:
df_train[4:5]

,068/09_068.png,068_forg/03_0113068.PNG,1
4,068/09_068.png,068_forg/04_0113068.PNG,1


In [11]:
image1_path=os.path.join(train_dir,df_train.iat[4,0])
image1_path

'/content/sign_data/train/068/09_068.png'

In [12]:
class Sign_Data(Dataset):
    def __init__(self,train_dir=None,train_csv=None,transform=None):
        self.train_dir=train_dir
        self.train_data=pd.read_csv(train_csv)
        self.train_data.columns=['image1','image2','class']
        self.transform=transform
        
    def __getitem__(self,idx): ## __getitem__ returns a sample data given index, idx=index
        
        img1_path=os.path.join(self.train_dir,self.train_data.iat[idx,0])
        img2_path=os.path.join(self.train_dir,self.train_data.iat[idx,1])
        
        img1=Image.open(img1_path)
        img2=Image.open(img2_path)
        
        img1=img1.convert('L') #L mode image, that means it is a single channel image - normally interpreted as greyscale.
        img2=img2.convert('L')
        
        img1=self.transform(img1)
        img2=self.transform(img2)
        
        return img1, img2, torch.from_numpy(np.array([int(self.train_data.iat[idx,2])],dtype=np.float32))
    
    
    def __len__(self): ## __len__ returns the size of the dataset..
        return len(self.train_data)

# Returns Image1, Image2 and the class label(whether 0 or 1).

In [13]:
dataset = Sign_Data(train_dir,train_csv,transform=transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()]))

In [14]:
dataset

# Siamese Network:-

In [15]:
class SiameseNetwork(nn.Module):
    
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        
        
        self.conv1=nn.Conv2d(1,50,kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        # L1 ImgIn shape=(?, 28, 28, 1)      # (n-f+2*p/s)+1
        #    Conv     -> (?, 24, 24, 50)
        #    Pool     -> (?, 12, 12, 50)
        
        
        self.conv2 = nn.Conv2d(50,60, kernel_size = 5)
        self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2, padding = 0)
        # L2 ImgIn shape=(?, 12, 12, 50)
        #    Conv      ->(?, 8, 8, 60)
        #    Pool      ->(?, 4, 4, 60)
        
        
        self.conv3 = nn.Conv2d(60, 80,  kernel_size = 3)
        # L3 ImgIn shape=(?, 4, 4, 60)
        #    Conv      ->(?, 2, 2, 80)
        
        self.batch_norm1 = nn.BatchNorm2d(50)
        self.batch_norm2 = nn.BatchNorm2d(60)
        
        #self.dropout1 = nn.Dropout2d()
        
        # L4 FC 2*2*80 inputs -> 250 outputs
        self.fc1 = nn.Linear(32000, 128) 
        self.fc2 = nn.Linear(128, 2)
        
      
    
    
    def forward1(self,x):
        x=self.conv1(x)
        x = self.batch_norm1(x)
        x=F.relu(x)
        x=self.pool1(x)
        
        x=self.conv2(x)
        x = self.batch_norm2(x)
        x=F.relu(x)
        x=self.pool2(x)
        
        x=self.conv3(x)
        x=F.relu(x)
#         print(x.size())
        x = x.view(x.size()[0], -1)
#         print('Output2')
#         print(x.size()) #32000 thats why the input of fully connected layer is 32000
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x
    

    def forward(self, input1, input2):
        # forward pass of input 1
        output1 = self.forward1(input1)
        # forward pass of input 2
        output2 = self.forward1(input2)
        
        return output1, output2


# Constrastive Loss Function:-

Contrastive loss:-
Contrastive loss is widely-used in unsupervised and self-supervised learning. Originally developed by Hadsell et al. in 2016 from Yann LeCun’s group, this loss function operates on pairs of samples instead of individual samples. It defines a binary indicator Y for each pair of samples stating whether they should be deemed similar, and a learnable distance function D_W(x_1, x_2) between a pair of samples x_1, x_2, parameterized by the weights W in the neural network. 

, where m>0 is a margin. The margin defines a radius around the embedding space of a sample so that dissimilar pairs of samples only contribute to the contrastive loss function if the distance D_W is within the margin.


Intuitively, this loss function encourages the neural network to learn a embedding to place samples with the same labels close to each other, while distancing the samples with different labels in the embedding space.

In [16]:
class ContrastiveLoss(torch.nn.Module):
    
    def __init__(self, margin=1.5):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive
    

In [17]:
train_dataloader = DataLoader(dataset,
                        shuffle=True,
                        num_workers=8,
                        batch_size=32)

In [18]:
train_dataloader

In [19]:
if torch.cuda.is_available():
    print('Yes')

Yes


In [20]:
device = torch.device("cuda")
net = SiameseNetwork().to(device)

criterion = ContrastiveLoss()               
#optimizer = torch.optim.SGD(net.parameters(), lr = 3e-4) 

optimizer = optim.RMSprop(net.parameters(), lr=1e-4, alpha=0.99)

In [21]:
def train():
    loss= [] 

    for epoch in range(1,10):
        for i, data in enumerate(train_dataloader,0):
            img0, img1 , label = data
            img0, img1 , label = img0.cuda(), img1.cuda() , label.cuda()
            optimizer.zero_grad()
            output1,output2 = net(img0,img1)
            loss_contrastive = criterion(output1,output2,label)
            loss_contrastive.backward()
            optimizer.step()
            
        print("Epoch {}\n Current loss {}\n".format(epoch,loss_contrastive.item()))

        loss.append(loss_contrastive.item())
        
    return net

In [22]:
model = train()
torch.save(model.state_dict(), "model.pt")
print("Model Saved Successfully")

Epoch 1
 Current loss 0.4805964529514313

Epoch 2
 Current loss 0.5965831875801086

Epoch 3
 Current loss 0.5849786400794983

Epoch 4
 Current loss 0.5848731994628906

Epoch 5
 Current loss 0.5773109793663025

Epoch 6
 Current loss 0.5968514680862427

Epoch 7
 Current loss 0.5285646319389343

Epoch 8
 Current loss 0.6682752966880798

Epoch 9
 Current loss 0.631907045841217

Model Saved Successfully


In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SiameseNetwork().to(device)
model.load_state_dict(torch.load("model.pt"))


<All keys matched successfully>

In [24]:


test_dataset = Sign_Data(test_dir,test_csv,transform=transforms.Compose([transforms.Resize((100,100)),transforms.ToTensor()]))

test_dataloader = DataLoader(test_dataset,num_workers=8,batch_size=1,shuffle=True)

In [25]:
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()    

# Testing the model by comparing the model's distance prediction between two pairs of Signature:-

In [1]:
count=0
for i, data in enumerate(test_dataloader,0): 
  x0, x1 , label = data
  concat = torch.cat((x0,x1),0)
  output1,output2 = model(x0.to(device),x1.to(device))

  eucledian_distance = F.pairwise_distance(output1, output2)
    
  if label==torch.FloatTensor([[0]]):
    label="Original Pair Of Signature"
  else:
    label="Forged Pair Of Signature"
    
  imshow(torchvision.utils.make_grid(concat))
  print("Predicted Eucledian Distance:-",eucledian_distance.item())
  print("Actual Label:-",label)
  count=count+1
  if count ==10:
     break

NameError: ignored

# Mostly our model is predicting the distances quite well.